In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential, losses, metrics, optimizers, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
training_samples = 28317
validation_samples = 3541
training_size = training_samples + validation_samples

image_path = "../dataset/FERPlus/images.npy"
images = np.float32(np.load(image_path))
images = images/255.0
images = tf.convert_to_tensor(images)
images = tf.image.grayscale_to_rgb(images)
training_images = images[:training_size]
test_images = images[training_size:]

emotion_path = "../dataset/FERPlus/emotions_multi.npy"
emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, loss_fn, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
                  loss=loss_fn, 
                  metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

num_epochs = 40
loss_fns = [
    losses.SquaredHinge(),
    losses.Poisson(),
    losses.MeanSquaredLogarithmicError(),
    losses.MeanSquaredError(),
    losses.MeanAbsolutePercentageError(),
    losses.MeanAbsoluteError(),
    losses.LogCosh(),
    losses.KLDivergence(),
    losses.Huber(),
    losses.Hinge(),
    losses.CosineSimilarity(axis=1),
    losses.CategoricalHinge()
]
history_save_paths = [
    './results/SquaredHinge.txt',
    './results/Poisson.txt',
    './results/MeanSquaredLogarithmicError.txt',
    './results/MeanSquaredError.txt',
    './results/MeanAbsolutePercentageError.txt',
    './results/MeanAbsoluteError.txt',
    './results/LogCosh.txt',
    './results/KLDivergence.txt',
    './results/Huber.txt',
    './results/Hinge.txt',
    './results/CosineSimilarity.txt',
    './results/CategoricalHinge.txt'
]
batch_size = 32
val_accs = []
for i in range(len(loss_fns)):
    model = create_model()
    loss_fn = loss_fns[i]
    history_save_path = history_save_paths[i]
    history = train(model, loss_fn, num_epochs, batch_size)
    val_accs.append(history.history['val_model_acc'])
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)